# XGBoost For Glaucoma Surgery Prediction
Model construction, training, hyperparameter tuning, and evaluation for the XGBoost Fusion Model. The same process was applied for XGBoost single modality model training and evaluation as well. 

In [1]:
# imports
import sklearn 
import math 
import matplotlib.pyplot as plt
import lazypredict
import pandas as pd
import numpy as np
import os 
from sklearn.model_selection import train_test_split
from google.cloud import bigquery
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import RandomOverSampler
from sklearn.model_selection import RandomizedSearchCV
from imblearn.pipeline import Pipeline
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.metrics import f1_score
import xgboost as xgb
from sklearn.metrics import roc_curve, auc
import joblib

In [2]:
print(sklearn.__version__)


1.3.0


In [3]:
# F1 tuning function
def f1_tuning(test_pred,y_val):
    f1scores={}
    for i in np.arange(0.05,1.0,0.05):
        new_pred = np.where(test_pred<i, 0 , 1)
        f1scores[i]=f1_score(y_val,new_pred)
        
    return f1scores,max(f1scores, key=f1scores.get)

### Import Data

In [4]:
# Import single modality or fusion data 

# RNFL data
rnfl_train = pd.read_csv('data/rnfl_train.csv')
rnfl_val = pd.read_csv('data/rnfl_val.csv')
rnfl_test = pd.read_csv('data/rnfl_test.csv')

# Fusion data
rnflehr_train = pd.read_csv('data/rnflehr_train.csv')
rnflehr_val = pd.read_csv('data/rnflehr_val.csv')
rnflehr_test = pd.read_csv('data/rnflehr_test.csv')

# EHR data
ehr_train = pd.read_csv('data/ehr_train.csv')
ehr_val = pd.read_csv('data/ehr_val.csv')
ehr_test = pd.read_csv('data/ehr_test.csv')

In [5]:
X_train = rnfl_train.drop(columns=['target', 'pat_mrn'])
X_val = rnfl_val.drop(columns=['target', 'pat_mrn'])
X_test = rnfl_test.drop(columns=['target', 'pat_mrn'])

y_train = rnfl_train['target']
y_val = rnfl_val['target']
y_test = rnfl_test['target']

### XGB Fusion Model

In [2]:
# Prep data
X_train = rnflehr_train.drop(columns=['target', 'pat_mrn'])
X_val = rnflehr_val.drop(columns=['target', 'pat_mrn'])
X_test = rnflehr_test.drop(columns=['target', 'pat_mrn'])

y_train = rnflehr_train['target']
y_val = rnflehr_val['target']
y_test = rnflehr_test['target']

# Apply SMOTE to the training set to balance class distribution
smote = SMOTE(random_state=42)  
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

# Fit XGB 
xgb_clf = xgb.XGBClassifier()
xgb_clf.fit(X_train_resampled, y_train_resampled)

print(xgb_clf)

expected_y  = y_val
predicted_y = xgb_clf.predict(X_val)

print(metrics.classification_report(expected_y, predicted_y))
print(metrics.confusion_matrix(expected_y, predicted_y))

print('Valid',roc_auc_score(expected_y,predicted_y))

# Hyperparameter tuning
param_grid_xgb = {
    'n_estimators': [50, 100, 150],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 4, 5]
}

# Perform hyperparameter using the validation set
grid_search_xgb = GridSearchCV(xgb_clf, param_grid_xgb, scoring='f1')
grid_search_xgb.fit(X_val, y_val)

best_xgb_model = grid_search_xgb.best_estimator_

y_pred_xgb = best_xgb_model.predict(X_val)

yval_probs_xgb = best_xgb_model.predict_proba(X_val)
print('XGBoost validation roc-auc: ', roc_auc_score(y_val, yval_probs_xgb[:,1]))
print()

report_xgb = classification_report(y_val, y_pred_xgb)
print("Classification Report for XGBoost on Validation Set:")
print(report_xgb)

best_params_xgb = grid_search_xgb.best_params_
print("Best Hyperparameters for XGBoost:", best_params_xgb)

# F1 thresholding 
best_threshold = 0.5  # Starting threshold

best_f1_score = 0
optimal_threshold = best_threshold

for threshold in [0.3, 0.4, 0.5, 0.6, 0.7]:
    y_pred_thresholded = (yval_probs_xgb[:, 1] > threshold).astype(int)
    f1 = f1_score(y_val, y_pred_thresholded)
    
    if f1 > best_f1_score:
        best_f1_score = f1
        optimal_threshold = threshold

print("Optimal Threshold:", optimal_threshold)
print("Best F1 Score:", best_f1_score)

# Eval on test set
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, roc_auc_score, roc_curve

# XGBoost model
model = best_xgb_model  # Replace with your XGBoost model
model_name = "XGBoost"
dataset_name = "Fusion"
# optimal_threshold = optimal_t  # Replace with your optimal threshold

# Predict probabilities for the positive class on the test set
y_pred_probs_test = model.predict_proba(X_test)[:, 1]

# Calculate ROC AUC score
roc_auc = roc_auc_score(y_test, y_pred_probs_test)

# Generate classification report
y_pred_thresholded = (y_pred_probs_test > optimal_threshold).astype(int)
report = classification_report(y_test, y_pred_thresholded, digits=3)
print(f"Classification Report for {model_name} on {dataset_name} Test Set:")
print(report)

# Calculate ROC curve
fpr, tpr, _ = roc_curve(y_test, y_pred_probs_test)

# Plot ROC curve for the XGBoost model
plt.figure()
plt.plot(fpr, tpr, label=f'{model_name} (AUC = {roc_auc:.3f})')
plt.plot([0, 1], [0, 1], color='gray', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title(f'Receiver Operating Characteristic - {model_name}')
plt.legend(loc='lower right')
plt.show()


In [19]:
# Save model

import joblib

model = best_xgb_model  
model_filename = 'models/xgboost/xgboost_fusion_glauc_model.pkl'
joblib.dump(model, model_filename)

print(f"Model saved to {model_filename}")

Model saved to models_updated/xgboost/xgboost_fusion_glauc_model.pkl
